In [ ]:

1.Listázzuk azon vendégek nevét, email-címét és felhasználói nevét, akik egynél többször foglaltak!

Hagyjuk ki azokat a vendégeket, akik május hónapban születtek!
Válaszszöveg 1 kérdés
Rich text szerkesztő


In [ ]:
SELECT V.NEV, V.EMAIL, V.USERNEV
FROM VENDEG V
JOIN FOGLALAS F ON V.USERNEV = F.UGYFEL_FK
WHERE MONTH(V.SZUL_DAT) <> 5
GROUP BY V.NEV, V.EMAIL, V.USERNEV
HAVING COUNT(F.FOGLALAS_PK) > 1;

2.Listázzuk azon szálláshelyek azonosítóját és nevét, amelyek a saját helyükön (HELY) a legtöbb csillaggal rendelkeznek!

In [ ]:
SELECT sz.SZALLAS_ID, sz.SZALLAS_NEV
FROM Szallashely sz
JOIN 
    (SELECT 
         HELY,
         MAX(CSILLAGOK_SZAMA) AS max_csillagok
     FROM 
         Szallashely
     WHERE
         CSILLAGOK_SZAMA > 0
     GROUP BY 
         HELY) max_szallas ON sz.HELY = max_szallas.HELY AND sz.CSILLAGOK_SZAMA = max_szallas.max_csillagok

3.Jelenítsük meg, hogy az egyes szálláshelyek szobáit esetén a legutolsó 5 foglalás átlagosan hány főre történt!

A számításnál vegyük figyelembe a felnőtteket és a gyermekeket is
A foglalás dátumánál a METTOL mezőt használjuk



In [ ]:
WITH UtolsoOT AS (
    SELECT 
        SZOBA_FK, 
        GYERMEK_SZAM + FELNOTT_SZAM AS OSSZESEN,
        ROW_NUMBER() OVER (PARTITION BY SZOBA_FK ORDER BY METTOL DESC) AS f
    FROM FOGLALAS )
SELECT 
    SZOBA_FK, 
    AVG(OSSZESEN) AS ATLAGOSAN
FROM UtolsoOT
WHERE f <= 5
GROUP BY SZOBA_FK;

4.Készítsen lekérdezést, amely megjeleníti, hogy szálláshelyenként, azon belül a pótágyak száma szerint hány főre foglaltak összesen (felnőtt + gyermek)!

Jelenjenek meg a részösszegek és a végösszeg is!


In [ ]:
SELECT 
    SZALLASHELY.SZALLAS_NEV,
    SZOBA.POTAGY,
    SUM(FOGLALAS.FELNOTT_SZAM + FOGLALAS.GYERMEK_SZAM) AS OSSZES_FO
FROM 
    SZALLASHELY
JOIN 
    SZOBA ON SZALLASHELY.SZALLAS_ID = SZOBA.SZALLAS_FK
JOIN 
    FOGLALAS ON SZOBA.SZOBA_SZAMA = FOGLALAS.SZOBA_FK
GROUP BY 
    ROLLUP(SZALLASHELY.SZALLAS_NEV, SZOBA.POTAGY);



5.A Szoba tábla Ferohely oszlopára végezzük el a következő számításokat:

Különböző értékek száma
Minimum, maximum, átlag
Null és Not null értékek százalékos aránya

In [ ]:
SELECT 
    COUNT(DISTINCT FEROHELY) AS KulonbozoErtekek,
    MIN(FEROHELY) AS Minimum,
    MAX(FEROHELY) AS Maximum,
    AVG(FEROHELY) AS Atlag,
    (COUNT(FEROHELY) * 100.0 / COUNT(*)) AS NemNullaSzázalék,
    ((COUNT(*) - COUNT(FEROHELY)) * 100.0 / COUNT(*)) AS NullaSzázalék
FROM 
    SZOBA;


6.Valósítsa meg REDIS-ben a következő táblákat:

IRO(ID, nev, nemzetiseg)
KONYV(ID, cim, mufaj, IRO_ID)
Vigyen fel táblánként legalább 2-2 sornyi adatot!

In [ ]:
HSET IRO:1 ID 1 nev "József Attila" nemzetiseg "Magyar"
HSET IRO:2 ID 2 nev "Petőfi Sándor" nemzetiseg "Magyar"
HSET KONYV:1 ID 1 cim "Harry Potter" mufaj "Kaland" IRO_ID 1
HSET KONYV:2 ID 2 cim "Dune" mufaj "Sci-fi" IRO_ID 2

7.Cassandra CQL Shellben hozza létre a következő táblát:

STUDENT(ID, Nev, Teljesitett_felevek, Eredmenyek, Nyelvtudas)

Az ID legyen az elsődleges kulcs
A Teljesitett félévek legyen LIST, tartalma egész számok, pl: 1, 2, 3
Az Eredmenyek legyen MAP, tartalma, hogy melyik félévben mennyi volt az átlag, pl: 1: 3,00, 2: 4.55, 3: 4,10
A Nyelvtudas legyen SET, tartalma a nyelvek megnevezése
Vigyen fel legalább 2 rekordot!

In [ ]:
CREATE TABLE STUDENT (
    ID int PRIMARY KEY,
    Nev text,
    Teljesitett_felevek list<int>,
    Eredmenyek map<int, float>,
    Nyelvtudas set<text>
);

In [ ]:
INSERT INTO STUDENT (ID, Nev, Teljesitett_felevek, Eredmenyek, Nyelvtudas) 
VALUES (1, 'John Doe', [1, 2, 3], {1: 3.00, 2: 4.55, 3: 4.10}, {'English'});

INSERT INTO STUDENT (ID, Nev, Teljesitett_felevek, Eredmenyek, Nyelvtudas) 
VALUES (2, 'Péter Zsombor', [1, 2, 3, 4], {1: 3.75, 2: 3.85, 3: 4.25, 4: 3.95}, {'Hungarian', 'French'});

8.A Neo4j Sandbox-ban vagy a Desktopban jelenítsük meg azon személyeket, akik egy filmben szerepeltek Tom Hanks-sel.

Egy szereplő neve csak egyszer jelenjen meg
Tom Hanks ne legyen benne a listában

In [ ]:
MATCH (p1:Person)-[:ACTED_IN]->(:Movie)<-[:ACTED_IN]-(p2:Person)
WHERE p1.name = 'Tom Hanks' AND p1 <> p2
RETURN DISTINCT p2.name AS Színész